In [2]:
import numpy as np
import pandas as pd

In [11]:
import sys
#sys.path.append("..")
from simdb.databaseModel import *
from simdb.databaseAPI import *

ImportError: No module named simdb.databaseModel

In [4]:
# db_path = 'test_WORKS.db'
db_path = '/home/andrejb/Research/SIMS.db'

## For migration to databaseAPI:

## HowTo

In [ ]:
# db = openDatabase(db_path) # open database
# one should not keep the DB open

In [5]:
getTags(db_path)  # get all tags which are used in DB

array([u'test', u'test2'], dtype='<U5')

In [6]:
keywords = getKeywords(db_path)  # get all keyword with the values they have
pd.Series(keywords) 

chainlen                                                      [1, 2]
expansion_from     [2016_08_09_2ub_k11_01, 2016_08_09_2ub_k11_02,...
expansion_point    [x_-0.11_y_-4.63, x_-0.18_y_-5.01, x_-0.20_y_-...
expansion_type                                  [cg_min, cg_min_rnd]
extension_from     [2016_08_09_2ub_k11_01, 2016_08_09_2ub_k11_02,...
extension_type                                              [middle]
ff                                                 [GROMOS, MARTINI]
linker                        [k11, k27, k29, k33, k48, k6, k63, m1]
min                                                             [01]
mutation              [K11C, K27C, K29C, K33C, K48C, K63C, K6C, nat]
n_chains                                                      [1, 2]
number             [0, 01, 02, 03, 04, 05, 06, 07, 1, 10, 11, 12,...
pH                                                            [5, 7]
protein                                          [FAT10, UBQ, diUBQ]
seq                 [CD_exp, CD_na

In [7]:
table = getEntryTable(db_path) # load whole table

### Selection

In [8]:
table[selectByTag(table, "test")] # select by tag

,entry_id,path,chainlen,expansion_from,expansion_point,expansion_type,extension_from,extension_type,ff,linker,...,n_chains,number,pH,protein,seq,src,structure,stype,temp,tags
124,2018_04_06_2xUb_unlinked_48,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,"test,test2"
585,2018_04_06_2xUb_unlinked_47,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,test


In [9]:
table[selectByKeyword(table, "ff", "MARTINI")] # select by keyword

,entry_id,path,chainlen,expansion_from,expansion_point,expansion_type,extension_from,extension_type,ff,linker,...,n_chains,number,pH,protein,seq,src,structure,stype,temp,tags
7,2018_04_06_2xUb_unlinked_22,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
9,2017_07_21_2ub_k11_02_02,/home/andrejb/Research/SIMS/2017_07_21_2ub_k11...,2,None,None,None,None,None,MARTINI,k11,...,None,02,None,UBQ,None,None,02,None,None,NaN
14,2016_08_09_2ub_k11_01,/home/andrejb/Research/SIMS/2016_08_09_2ub_k11_01,2,None,None,None,None,None,MARTINI,k11,...,None,01,None,UBQ,None,None,01,None,None,NaN
16,2017_04_26_M_ub1_300K,/home/andrejb/Research/SIMS/2017_04_26_M_ub1_300K,1,None,None,None,None,None,MARTINI,None,...,1,None,None,UBQ,None,None,None,None,300,NaN
17,2017_08_01_2ub_k48_01_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,2,None,None,None,None,None,MARTINI,k48,...,None,06,None,UBQ,None,None,01,None,None,NaN
18,2017_08_01_2ub_k29_01_03,/home/andrejb/Research/SIMS/2017_08_01_2ub_k29...,2,None,None,None,None,None,MARTINI,k29,...,None,03,None,UBQ,None,None,01,None,None,NaN
21,2018_04_06_2xUb_unlinked_23,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
23,2017_08_01_2ub_k11_02_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k11...,2,None,None,None,None,None,MARTINI,k11,...,None,06,None,UBQ,None,None,02,None,None,NaN
24,2018_04_06_2xUb_unlinked_34,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
32,2016_11_26_2ub_k27_04,/home/andrejb/Research/SIMS/2016_11_26_2ub_k27_04,2,None,None,None,2016_08_09_2ub_k27_01,middle,MARTINI,k27,...,None,02,None,UBQ,None,None,None,extension,None,NaN


In [10]:
# combinations of selections
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "1")
tags     = selectByTag(table, "test2")

table[ff & chainlen & tags]

,entry_id,path,chainlen,expansion_from,expansion_point,expansion_type,extension_from,extension_type,ff,linker,...,n_chains,number,pH,protein,seq,src,structure,stype,temp,tags
124,2018_04_06_2xUb_unlinked_48,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,"test,test2"
132,2018_04_06_2xUb_unlinked_46,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,test2


In [11]:
# another example
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "2")
linker1  = selectByKeyword(table, "linker", "k48")
linker2  = selectByKeyword(table, "linker", "k63")

table[ff & chainlen & (linker1 | linker2)]

,entry_id,path,chainlen,expansion_from,expansion_point,expansion_type,extension_from,extension_type,ff,linker,...,n_chains,number,pH,protein,seq,src,structure,stype,temp,tags
17,2017_08_01_2ub_k48_01_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,2,None,None,None,None,None,MARTINI,k48,...,None,06,None,UBQ,None,None,01,None,None,NaN
65,2017_08_01_2ub_k48_02_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,2,None,None,None,None,None,MARTINI,k48,...,None,06,None,UBQ,None,None,02,None,None,NaN
78,2017_08_01_2ub_k48_02_03,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,2,None,None,None,None,None,MARTINI,k48,...,None,03,None,UBQ,None,None,02,None,None,NaN
113,2016_08_09_2ub_k63_01,/home/andrejb/Research/SIMS/2016_08_09_2ub_k63_01,2,None,None,None,None,None,MARTINI,k63,...,None,01,None,UBQ,None,None,01,None,None,NaN
134,2017_07_21_2ub_k63_01_02,/home/andrejb/Research/SIMS/2017_07_21_2ub_k63...,2,None,None,None,None,None,MARTINI,k63,...,None,02,None,UBQ,None,None,01,None,None,NaN
162,2017_07_21_2ub_k48_01_02,/home/andrejb/Research/SIMS/2017_07_21_2ub_k48...,2,None,None,None,None,None,MARTINI,k48,...,None,02,None,UBQ,None,None,01,None,None,NaN
181,2016_11_26_2ub_k48_04,/home/andrejb/Research/SIMS/2016_11_26_2ub_k48_04,2,None,None,None,2016_08_09_2ub_k48_01,middle,MARTINI,k48,...,None,02,None,UBQ,None,None,None,extension,None,NaN
197,2017_07_21_2ub_k48_02_02,/home/andrejb/Research/SIMS/2017_07_21_2ub_k48...,2,None,None,None,None,None,MARTINI,k48,...,None,02,None,UBQ,None,None,02,None,None,NaN
198,2016_08_09_2ub_k63_02,/home/andrejb/Research/SIMS/2016_08_09_2ub_k63_02,2,None,None,None,None,None,MARTINI,k63,...,None,01,None,UBQ,None,None,02,None,None,NaN
215,2016_11_26_2ub_k48_05,/home/andrejb/Research/SIMS/2016_11_26_2ub_k48_05,2,None,None,None,2016_08_09_2ub_k48_02,middle,MARTINI,k48,...,None,03,None,UBQ,None,None,None,extension,None,NaN


### Running functions

## Deva

* Add tags/keywords to all simulations in pandas table
* Add simulations to DB from pandas table